In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
649,There really isn't much to say about this movi...,positive
974,This movie is about a very delicate argument a...,positive
836,"I know it's not original, but what the hey? Wh...",negative
717,Having majored in Western History when I was a...,positive
983,There is no way to put into words just how bad...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
649,really much say movie it s crude fun br br plo...,positive
974,movie delicate argument searching something ma...,positive
836,know original hey else said it feel unutterabl...,negative
717,majored western history student college seeing...,positive
983,way put word bad shapeless paceless laughless ...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    254
positive    246
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
649,really much say movie it s crude fun br br plo...,1
974,movie delicate argument searching something ma...,1
836,know original hey else said it feel unutterabl...,0
717,majored western history student college seeing...,1
983,way put word bad shapeless paceless laughless ...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [12]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/PrathameshDevkar/mlops_capstone_project_movie_recommender_system.mlflow')
dagshub.init(repo_owner='PrathameshDevkar', repo_name='mlops_capstone_project_movie_recommender_system', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Prathamesh\MLOPS\mlops_capstone_project_movie_recommender_system\atlas\lib\site-packages\rich\live.py:256:
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=8b132f0c-c757-4649-bf72-b0cba7914da6&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=33448ee2c4addb8bc1cd3ad69fb63516aff77e908d2a8320775b2515a8a27302




Accessing as PrathameshDevkar

Initialized MLflow to track repo "PrathameshDevkar/mlops_capstone_project_movie_recommender_system"

Repository PrathameshDevkar/mlops_capstone_project_movie_recommender_system initialized!

2025/08/01 15:17:50 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/c53d94afb2d14928a97dffd611c8ee4a', creation_time=1754041635496, experiment_id='0', last_update_time=1754041635496, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [15]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "countvectorizer/Bag of Words")
        mlflow.log_param("num_features", 40)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        joblib.dump(model,'model.pkl')
        mlflow.log_artifact('model.pkl', artifact_path='model')
        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-01 15:52:04,958 - INFO - Starting MLflow run...
2025-08-01 15:52:05,438 - INFO - Logging preprocessing parameters...
2025-08-01 15:52:06,490 - INFO - Initializing Logistic Regression model...
2025-08-01 15:52:06,493 - INFO - Fitting the model...
2025-08-01 15:52:06,523 - INFO - Model training complete.
2025-08-01 15:52:06,526 - INFO - Logging model parameters...
2025-08-01 15:52:06,873 - INFO - Making predictions...
2025-08-01 15:52:06,875 - INFO - Calculating evaluation metrics...
2025-08-01 15:52:06,891 - INFO - Logging evaluation metrics...
2025-08-01 15:52:11,416 - INFO - Saving and logging the model...
2025-08-01 15:52:13,568 - INFO - Model training and logging completed in 8.13 seconds.
2025-08-01 15:52:13,571 - INFO - Accuracy: 0.61
2025-08-01 15:52:13,571 - INFO - Precision: 0.6538461538461539
2025-08-01 15:52:13,572 - INFO - Recall: 0.6181818181818182
2025-08-01 15:52:13,573 - INFO - F1 Score: 0.6355140186915887


🏃 View run abrasive-goat-195 at: https://dagshub.com/PrathameshDevkar/mlops_capstone_project_movie_recommender_system.mlflow/#/experiments/0/runs/602c1c93e0fe4979855f6c1a9baeba1c
🧪 View experiment at: https://dagshub.com/PrathameshDevkar/mlops_capstone_project_movie_recommender_system.mlflow/#/experiments/0
